<a id='pred_top'>

# Predict auction price

Try several models and improve predicition accuracy

## Model fitting

- Linear fits  
  1. [Simple linear fit](#pred_model_1)  
     No cross validation. Observations with missing values are dropped.
  2. [Dependent values scaled](#pred_model_2)  
     Dependent value here is _prices_.
  3. [Partial data](#pred_model_3)  
     Only young cars
- Multiple linear regression models  
  1. [MLR fit without imputation](#pred_model_4)  
  2. [With imputation](#pred_model_5)  
  3. [Include categorical features](#pred_model_6)  
  4. [Lasso regularization](#pred_model_7)  
  5. [include engineered features](#pred_model_8)

## Results

- [Model performance](#pred_accuracies)
- [Save best model](#pred_save_model) **TODO**  
  This is not implemented yet. Some preprocessing functions are not handled well with `pickle`.
- [Predictions](#pred_predict)
     
  

In [1]:
import os
# setting path
os.chdir(r'..')

import drz_config
cfg = drz_config.read_config()
VERBOSE = cfg['VERBOSE']
SKIPSAVE = cfg['SKIPSAVE']

if VERBOSE > 0:
    display(cfg)

if cfg['OPBOD']:
    raise NotImplementedError

{'settings_fn': '../code/assets/drz-auction-settings.ini',
 'DATE': '2023-10',
 'VERBOSE': 1,
 'OPBOD': False,
 'URL': 'http://verkoop.domeinenrz.nl/verkoop_bij_inschrijving_2023-0010',
 'EXTEND_URL': False,
 'CLOSEDDATA': True,
 'closed_data_fields': '*',
 'SKIPSAVE': False}

In [2]:
# import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

import seaborn as sns

In [3]:
# set figure defaults (needs to be in cell seperate from import sns)
plt.style.use(['default', '../assets/movshon.mplstyle', '../assets/context-notebook.mplstyle'])

# Load data

In [4]:
fn = '../data/cars-for-ml.pkl'
print(fn)
df = pd.read_pickle(fn)
print(df.shape)

# time deltas
sel = df.dtypes == 'timedelta64[ns]'
df.loc[:, sel] = df.loc[:, sel].applymap(lambda x: x.days)
# nullable boolean
sel = df.dtypes == 'boolean'
df.loc[:,sel] = df.loc[:,sel].astype('O').fillna(np.NaN)
# int to float
df.price = df.price.astype(float)
# categories
cat_columns = ['brand', 'model', 'fuel', 'body_type','color', 'energy_label', 'fourwd', 'automatic_gearbox', 'under_survey']
# numerical
num_columns = list(np.setdiff1d(df.columns, cat_columns + ['price']))
df.loc[:, num_columns] = df.loc[:, num_columns].astype(float)

# Factorized categorical values
fld = 'energy_label'
# replace empty with NaN creates factor '-1'
v, idx = pd.factorize(df[fld].replace({'': np.NaN}), sort=True)
# convert '-1' back to NaN
v = v.astype(float)
v[v==-1] = np.NaN
# Store in dataframe
new_col = 'converted_' + fld
df[new_col] = v
# update list
num_columns += [new_col]
cat_columns.remove(fld)
print('\nCategorical field [{}] is converted to sequential numbers with: '.format(fld), end='\n\t')
print(*['{} <'.format(c) for c in idx], end='\n\n')

# convert boolean to string
for fld in ['fourwd', 'automatic_gearbox', 'under_survey']:
    if fld not in df.columns:
        print(f'!{fld} not in data!. Skip for now')
        continue
    new_col = fld
    # # update list
    # cat_columns += [new_col]
    # cat_columns.remove(fld)
    replace_dict = {
        '': '', 
        True: 'y', 
        False: 'n'
    }
    df[new_col] = df[fld].replace(replace_dict)
    print('\nBoolean field [{}] is converted to numbers according to: '.format(fld), end='\n')
    print(*['\t"{}" -> {} ({})\n'.format(k,v, type(v)) for k,v in replace_dict.items()], end='\n\n')

# convert integer to float and replace -1
for fld in ['number_of_cylinders', 'number_of_doors', 'number_of_gears', 'number_of_seats']:
    if fld not in df.columns:
        print(f'!{fld} not in data!. Skip for now')
        continue
    new_col = fld
    replace_dict = {
        -1: np.NaN, 
    }
    df[new_col] = df[fld].replace(replace_dict).astype(float)

# convert empty string to NaN
for fld in ['brand', 'model', 'fuel', 'body_type', 'color', 'fourwd']:
    if fld not in df.columns:
        print(f'!{fld} not in data!. Skip for now')
        continue
    new_col = fld
    replace_dict = {
        '': np.NaN, 
    }
    df[new_col] = df[fld].replace(replace_dict)

# translate Dutch to English
fld = 'color'
new_col = fld
# # update list
# cat_columns += [new_col]
# cat_columns.remove(fld)
replace_dict = {
    '': 'missing', 
    'BLAUW': 'Blue',
    'ROOD': 'Red',
    'GROEN': 'Green',
    'GRIJS': 'Gray',
    'WIT': 'White',
    'ZWART': 'Black',
    'BEIGE': 'Beige',
    'BRUIN': 'Brown',
    'ROSE': 'Pink',
    'GEEL': 'Yellow',
    'CREME': 'Creme',
    'ORANJE': 'Orange',
    'PAARS': 'Purple,'
}
df[new_col] = df[fld].replace(replace_dict)
print('\nField [{}] is converted according to: '.format(fld), end='\n')
print(*['\t"{}" -> {} ({})\n'.format(k,v, type(v)) for k,v in replace_dict.items()], end='\n\n')

# reporting
try:
    print('Categorical:', len(cat_columns))
    [print('\t[{:2.0f}] {:s}'.format(i+1, c)) for i,c in enumerate(df[cat_columns].columns)]
    print('Numercial:', len(num_columns))
    [print('\t[{:2.0f}] {:s}'.format(i+1, c)) for i,c in enumerate(df[num_columns].columns)]
    print('Last lot in data set:\n\t{}'.format(df.index[-1]))
except:
    cat_columns = [c for c in cat_columns if c in df.columns]
    num_columns = [c for c in num_columns if c in df.columns]    
    print('! not all fields are in data !. Skip for now')

../data/cars-for-ml.pkl
(10321, 29)

Categorical field [energy_label] is converted to sequential numbers with: 
	A < B < C < D < E < F < G < nan <


Boolean field [fourwd] is converted to numbers according to: 
	"" ->  (<class 'str'>)
 	"True" -> y (<class 'str'>)
 	"False" -> n (<class 'str'>)



Boolean field [automatic_gearbox] is converted to numbers according to: 
	"" ->  (<class 'str'>)
 	"True" -> y (<class 'str'>)
 	"False" -> n (<class 'str'>)



Boolean field [under_survey] is converted to numbers according to: 
	"" ->  (<class 'str'>)
 	"True" -> y (<class 'str'>)
 	"False" -> n (<class 'str'>)



Field [color] is converted according to: 
	"" -> missing (<class 'str'>)
 	"BLAUW" -> Blue (<class 'str'>)
 	"ROOD" -> Red (<class 'str'>)
 	"GROEN" -> Green (<class 'str'>)
 	"GRIJS" -> Gray (<class 'str'>)
 	"WIT" -> White (<class 'str'>)
 	"ZWART" -> Black (<class 'str'>)
 	"BEIGE" -> Beige (<class 'str'>)
 	"BRUIN" -> Brown (<class 'str'>)
 	"ROSE" -> Pink (<class 'str'>)
 	"GE

In [5]:
# Store model results in dictonary: Instantiate empty dict
models = dict()

<a href="#pred_top" id='pred_model_1'><font size=+1><center>^^ TOP ^^</center></font></a>

---

# Model: Simple linear fit
Regress age (in days) with price (euro).  

## >> BIG FAT WARNING <<
All data is used without train / test split. I.e. accuracy is based on data that was used for fit. This is considered bad practice!

## Prepare input

In [6]:
from sklearn import linear_model

model_name = 'linear regression no cv'

X = df.dropna(subset=['price','age']).age.values.reshape(-1,1)
y = df.dropna(subset=['price','age']).price.values.reshape(-1,1)
print(X.shape)
print(y.shape)

(7514, 1)
(7514, 1)


## Fit

In [7]:
# instantiate a dict in models at key with name of this model
models[model_name] = dict()

# create regression model object and store
reg = linear_model.LinearRegression()
models[model_name].update({'model':reg})

# fit
reg.fit(X,y) # fit with all data
models[model_name].update({'n':y.shape[0]})

# parameters
betas = [*reg.intercept_, *reg.coef_[0]]
models[model_name].update({'betas':betas})

# score
models[model_name].update({'R^2':reg.score(X,y)})

In [8]:
# Fit a line by using predict
prediction_X = np.array([0,int(np.ceil(X.max()/365.25))*365.25]).reshape(-1,1)
prediction_y = reg.predict(prediction_X)

# plot
plt.figure(figsize=[8,8])
plt.plot(X/365.25, y/1000, marker='s', markeredgecolor = (0, 0, 0, 1), markerfacecolor = (1, 1, 1, .5), linestyle='None', ms=4)
hdl_fit = plt.plot(prediction_X/365, prediction_y/1000, color='blue', marker=None, linestyle='-', linewidth=4)
plt.legend(hdl_fit, ['n = {}, $R^2$ = {:.2f}\ny = {:+.0f}{:+.2f}*(x*365.25)'.format(
    models[model_name]['n'],
    models[model_name]['R^2'],
    *models[model_name]['betas']
)], loc='upper right')
plt.xlabel('Age (years)', style='italic')
plt.ylabel('Winning bid (EUR X1000)', style='italic')
plt.title('Simple linear fit', style='italic')
plt.ylim(bottom = -10)
plt.xlim(left = 0)

# Save
file_name = '../results/{}.png'.format(model_name.replace(' ','_'))
if (SKIPSAVE==False): #and (not(os.path.isfile(file_name))):
    print(file_name)
    with plt.style.context('../assets/context-paper.mplstyle'):
        plt.savefig(file_name, bbox_inches='tight', transparent=False)
else:
    plt.show()
    print(f'Skip. {file_name} exists or saving is disabled in settings.')

../results/linear_regression_no_cv.png


<a href="#pred_top" id='pred_model_2'><font size=+1><center>^^ TOP ^^</center></font></a>

---

## Model: linear but with scaled dependent values (prices)

Instead of using all data **train/test split** is performed. Also prices are log transformed.  

## Prepare input

In [9]:
from sklearn.model_selection import train_test_split, cross_val_score

model_name = 'linear regression log price'

X = df.dropna(subset=['price','age']).age.values.reshape(-1,1)
y = np.log10(df.dropna(subset=['price','age']).price.values.reshape(-1,1))
print(X.shape)
print(y.shape)

(7514, 1)
(7514, 1)


## Fit

In [10]:
# instantiate a dict in models at key with name of this model
models[model_name] = dict()

# split train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)
print(X_train.shape)
print(X_test.shape)

# create regression model object and store
reg = linear_model.LinearRegression()
models[model_name].update({'model':reg})

# fit
reg.fit(X_train,y_train) # fit with training set
models[model_name].update({'n':y.shape[0]})

# parameters
betas = [*reg.intercept_, *reg.coef_[0]]
models[model_name].update({'betas':betas})

# score
models[model_name].update({'R^2':reg.score(X,y)})
models[model_name].update({'test R^2':reg.score(X_test,y_test)})
cv_results = cross_val_score(reg, X_test, y_test, cv=5)
models[model_name].update({'cv R^2':cv_results})


(5259, 1)
(2255, 1)


In [11]:
depr_half_n_days = -(np.log10(2)/models[model_name]['betas'][1])
print('According to "{}"-model'.format(model_name))
print('Car depreciates to half its value every\n\t{:.0f} days ({:.1f} years).'.format(depr_half_n_days, depr_half_n_days/365.25))
for y in [0,2,4,6,8]:
    print('\ty(t={:+5.0f}) = {:.0f} euro'.format(y, 10**reg.predict([[y*365.25]])[0][0]))
print('\n\ty(t={:+5.1f}) = {:.0f} euro'.format(depr_half_n_days/365.25, 10**reg.predict([[depr_half_n_days]])[0][0]))
print('\ty(t=0) / 2 = {:.0f} euro'.format(10**models[model_name]['betas'][0]/2))

According to "linear regression log price"-model
Car depreciates to half its value every
	2281 days (6.2 years).
	y(t=   +0) = 11243 euro
	y(t=   +2) = 9005 euro
	y(t=   +4) = 7212 euro
	y(t=   +6) = 5776 euro
	y(t=   +8) = 4626 euro

	y(t= +6.2) = 5622 euro
	y(t=0) / 2 = 5622 euro


In [12]:
# Fit a line by using predict
prediction_X = np.array([0,int(np.ceil(X.max()/365.25))*365.25]).reshape(-1,1)
prediction_y = reg.predict(prediction_X)

# plot
plt.figure(figsize=[8,8])
hdl_trn = plt.plot(X_train/365.25, np.power(10,y_train), marker='s', markeredgecolor = (0, 0, 1, 1), markerfacecolor = (1, 1, 1, .5), linestyle='None', ms=4, 
                   label='train (n = {})'.format(y_train.shape[0]))
hdl_tst = plt.plot(X_test/365.25, np.power(10,y_test), marker='s', markeredgecolor = (0, 0, 0, 1), markerfacecolor = (1, 1, 1, .5), linestyle='None', ms=4, 
                   label='test (n = {}, $R^2$ = {:.2f})'.format(
                       y_test.shape[0],
                       models[model_name]['test R^2'],
                   ))
hdl_fit = plt.plot(prediction_X/365, np.power(10,prediction_y), color='blue', marker=None, linestyle='-', linewidth=4, 
                   label = '$log10(y)$ = {:+.2f}{:+.1e}*(x*365.25)\n($R^2$ = {:.2f}, $R^2_{{cv{:g}}}$ = {:.2f} (+/-{:.2f}))'.format(
                       *models[model_name]['betas'],
                       models[model_name]['R^2'],
                       models[model_name]['cv R^2'].shape[0],
                       np.mean(models[model_name]['cv R^2']),
                       np.std(models[model_name]['cv R^2']),
                   ))
plt.legend()
plt.xlabel('Age (years)', style='italic')
plt.ylabel('Winning bid (EUR)', style='italic')
plt.title('Linear fit with log(price)', style='italic')
plt.ylim(bottom = 10, top = 1000000)
plt.xlim(left = 0)
plt.yscale('log')

# Save
file_name = '../results/{}.png'.format(model_name.replace(' ','_'))
if (SKIPSAVE==False): #and (not(os.path.isfile(file_name))):
    print(file_name)
    with plt.style.context('../assets/context-paper.mplstyle'):
        plt.savefig(file_name, bbox_inches='tight', transparent=False)
else:
    plt.show()
    print(f'Skip. {file_name} exists or saving is disabled in settings.')

../results/linear_regression_log_price.png


<a href="#pred_top" id='pred_model_3'><font size=+1><center>^^ TOP ^^</center></font></a>

---

## Model: scaled price, but only young cars

Same as [model 2](#pred_model_2), but ignore cars older than 25y

## Prepare input

In [13]:
from sklearn.model_selection import train_test_split, cross_val_score

model_name = 'linear regression log price young'

is_yng = df.age/365.25 < 25

X = df[is_yng].dropna(subset=['price','age']).age.values.reshape(-1,1)
y = np.log10(df[is_yng].dropna(subset=['price','age']).price.values.reshape(-1,1))
print(X.shape)
print(y.shape)

(7343, 1)
(7343, 1)


## Fit

In [14]:
# instantiate a dict in models at key with name of this model
models[model_name] = dict()

# split train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)
print(X_train.shape)
print(X_test.shape)

# create regression model object and store
reg = linear_model.LinearRegression()
models[model_name].update({'model':reg})

# fit
reg.fit(X_train,y_train)
models[model_name].update({'n':y.shape[0]})

# parameters
betas = [*reg.intercept_, *reg.coef_[0]]
models[model_name].update({'betas':betas})

# score
models[model_name].update({'R^2':reg.score(X,y)})
models[model_name].update({'test R^2':reg.score(X_test,y_test)})
cv_results = cross_val_score(reg, X_test, y_test, cv=5)
models[model_name].update({'cv R^2':cv_results})


(5140, 1)
(2203, 1)


In [15]:
depr_half_n_days = -(np.log10(2)/models[model_name]['betas'][1])
print('According to "{}"-model'.format(model_name))
print('Car depreciates to half its value every\n\t{:.0f} days ({:.1f} years).'.format(depr_half_n_days, depr_half_n_days/365.25))
for y in [0,2,4,6,8]:
    print('\ty(t={:+5.0f}) = {:.0f} euro'.format(y, 10**reg.predict([[y*365.25]])[0][0]))
print('\n\ty(t={:+5.1f}) = {:.0f} euro'.format(depr_half_n_days/365.25, 10**reg.predict([[depr_half_n_days]])[0][0]))
print('\ty(t=0) / 2 = {:.0f} euro'.format(10**models[model_name]['betas'][0]/2))

According to "linear regression log price young"-model
Car depreciates to half its value every
	1352 days (3.7 years).
	y(t=   +0) = 24813 euro
	y(t=   +2) = 17061 euro
	y(t=   +4) = 11730 euro
	y(t=   +6) = 8065 euro
	y(t=   +8) = 5546 euro

	y(t= +3.7) = 12406 euro
	y(t=0) / 2 = 12406 euro


In [16]:
# Fit a line by using predict
prediction_X = np.array([0,int(np.ceil(X.max()/365.25))*365.25]).reshape(-1,1)
prediction_y = reg.predict(prediction_X)

# plot
plt.figure(figsize=[8,8])
hdl_trn = plt.plot(X_train/365.25, np.power(10,y_train), marker='s', markeredgecolor = (0, 0, 1, 1), markerfacecolor = (1, 1, 1, .5), linestyle='None', ms=4, 
                   label='train (n = {})'.format(y_train.shape[0]))
hdl_tst = plt.plot(X_test/365.25, np.power(10,y_test), marker='s', markeredgecolor = (0, 0, 0, 1), markerfacecolor = (1, 1, 1, .5), linestyle='None', ms=4, 
                   label='test (n = {}, $R^2$ = {:.2f})'.format(
                       y_test.shape[0],
                       models[model_name]['test R^2'],
                   ))
hdl_fit = plt.plot(prediction_X/365, np.power(10,prediction_y), color='blue', marker=None, linestyle='-', linewidth=4, 
                   label = '$log10(y)$ = {:+.2f}{:+.1e}*(x*365.25)\n($R^2$ = {:.2f}, $R^2_{{cv{:g}}}$ = {:.2f} (+/-{:.2f}))'.format(
                       *models[model_name]['betas'],
                       models[model_name]['R^2'],
                       models[model_name]['cv R^2'].shape[0],
                       np.mean(models[model_name]['cv R^2']),
                       np.std(models[model_name]['cv R^2']),
                   ))
plt.legend()
plt.xlabel('Age (years)', style='italic')
plt.ylabel('Winning bid (EUR)', style='italic')
plt.title('Linear fit with log(price) of young cars', style='italic')
plt.ylim(bottom = 10, top = 1000000)
plt.xlim(left = 0)
plt.yscale('log')

# Save
file_name = '../results/{}.png'.format(model_name.replace(' ','_'))
if (SKIPSAVE==False): #and (not(os.path.isfile(file_name))):
    print(file_name)
    with plt.style.context('../assets/context-paper.mplstyle'):
        plt.savefig(file_name, bbox_inches='tight', transparent=False)
else:
    plt.show()
    print(f'Skip. {file_name} exists or saving is disabled in settings.')

../results/linear_regression_log_price_young.png


<a href="#pred_top" id='pred_model_4'><font size=+1><center>^^ TOP ^^</center></font></a>

---

# Model: Multiple linear fit

Above [simple linear models](#pred_model_1) only use _Age_ as predictor of price. Here MLR will regress many (numerical) features with price (euro).  


## Prepare input

In [17]:
model_name = 'MLR reduced observations'

features = num_columns 
# Can be reduced here

X = df.dropna(subset=['price'] + features).loc[:,features].values.reshape(-1,len(features))
y = np.log10(df.dropna(subset=['price'] + features).price.values.reshape(-1,1))
print(X.shape)
print(y.shape)

(2087, 20)
(2087, 1)


## Fit

In [18]:
# instantiate a dict in models at key with name of this model
models[model_name] = dict()

# split train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)
print(X_train.shape)
print(X_test.shape)

# create regression model object and store
reg = linear_model.LinearRegression()
models[model_name].update({'model':reg})

# fit
reg.fit(X_train,y_train)
models[model_name].update({'n':y.shape[0]})
models[model_name].update({'n features':X.shape[1]})

# parameters
betas = [*reg.intercept_, *reg.coef_[0]]
models[model_name].update({'betas':betas})

# score
models[model_name].update({'R^2':reg.score(X,y)})
models[model_name].update({'test R^2':reg.score(X_test,y_test)})
cv_results = cross_val_score(reg, X_test, y_test, cv=5)
models[model_name].update({'cv R^2':cv_results})


(1460, 20)
(627, 20)


In [19]:
# plot coefficients
plt.figure(figsize=[8,2])
ax=plt.gca()

# sorted bar height
betas = models[model_name]['betas']
x = ['offset (log[EUR])'] + [features[i] for i in np.argsort(betas[1:])[::-1]]
y = [betas[0]] + sorted(betas[1:], reverse=True)

# plot bar
ax.bar(x=x, height=y, edgecolor='k', facecolor='None')

# add values when bar is small
for x_val, coef in zip(x,y):
    if np.abs(coef)<1:
        ax.text(x_val, coef, '{:.3g}'.format(coef), rotation=45, va='bottom', ha='left')
ax.set_yticks(range(0,5,2))

# plot origin
x_sign_switch = np.nonzero(np.array(y) < 0)[0][0]
ax.axvline(x_sign_switch-0.5, linewidth=2, linestyle='--', color='k')
ax.axhline(0, linewidth=2, linestyle='-', color='k')
        
x_sign_switch = np.nonzero(np.array(y) < 0)[0][0]
yl = ax.get_ylim()
ax.vlines(x_sign_switch-0.5, yl[0], yl[1], linewidth=2, linestyle='--')
ax.set_ylim(yl)
# ax.set_ylim(top=0.01, bottom=-0.01)

# labels
ax.set_xticks(x)
ax.set_xticklabels(labels=x, rotation=45, va='top', ha='right', style='italic')
ax.xaxis.set_tick_params(which='minor', bottom=False)
ax.set_xlabel('Feature', style='italic')
ax.set_ylabel('Coefficient (a.u.)', style='italic')
ax.set_title('Multiple linear regression', style='italic') 

# stats
xy=[ax.get_xlim()[1], ax.get_ylim()[1]]
ax.text(xy[0]*1.05,xy[1], '$R^2$ = {:.2f}, $R^2_{{cv{:g}}}$ = {:.2f} (+/-{:.2f})'.format(
    models[model_name]['R^2'],
    models[model_name]['cv R^2'].shape[0],
    np.mean(models[model_name]['cv R^2']),
    np.std(models[model_name]['cv R^2']),
) + '\n' +
         'train (n = {})'.format(y_train.shape[0]) + '\n' +
         'test (n = {}, $R^2$ = {:.2f})'.format(
             y_test.shape[0],
             models[model_name]['test R^2'],
         ), style='italic', va='top', ha='left')


# Save
file_name = '../results/{}.png'.format(model_name.replace(' ','_'))
if (SKIPSAVE==False): #and (not(os.path.isfile(file_name))):
    print(file_name)
    with plt.style.context('../assets/context-paper.mplstyle'):
        plt.savefig(file_name, bbox_inches='tight', transparent=False)
else:
    plt.show()
    print(f'Skip. {file_name} exists or saving is disabled in settings.')

../results/MLR_reduced_observations.png


<a href="#pred_top" id='pred_model_5'><font size=+1><center>^^ TOP ^^</center></font></a>

---

# Model: MLR + imputer

MLR as above, but instead of `dropna` us an imputer. This allows to use more observation.  

At this point a pipeline is used.

## Prepare input

In [20]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

model_name = 'MLR impute median'

features = num_columns 
# Can be reduced here

yX = df.loc[:,['price'] + features].dropna(subset=['price'])
X = yX.iloc[:,1:].values.reshape(-1,len(features))
y = np.log10(yX.iloc[:,0].values.reshape(-1,1))
print(X.shape)
print(y.shape)

(9019, 20)
(9019, 1)


## Fit

In [21]:
# instantiate a dict in models at key with name of this model
models[model_name] = dict()

# split train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)
print(X_train.shape)
print(X_test.shape)

# create regression model object and store
pl = make_pipeline(
    SimpleImputer(strategy='median'),
    StandardScaler(),
    linear_model.LinearRegression()
)
models[model_name].update({'model':pl})

# fit
pl.fit(X,y)
models[model_name].update({'n':y.shape[0]})
models[model_name].update({'n features':X.shape[1]})

# parameters
betas = [*pl.steps[-1][1].intercept_, *pl.steps[-1][1].coef_[0]]
models[model_name].update({'betas':betas})

# score
models[model_name].update({'R^2':pl.score(X,y)})
models[model_name].update({'test R^2':pl.score(X_test,y_test)})
cv_results = cross_val_score(pl, X_test, y_test, cv=5)
models[model_name].update({'cv R^2':cv_results})


(6313, 20)
(2706, 20)


In [22]:
# plot coefficients
plt.figure(figsize=[8,4])
ax = plt.gca()

# sorted bar height
betas = models[model_name]['betas']
x = ['offset (log[EUR])'] + [features[i] for i in np.argsort(betas[1:])[::-1]]
y = [betas[0]] + sorted(betas[1:], reverse=True)

# plot bar
ax.bar(x=x, height=y, edgecolor='k', facecolor='None')

# add values when bar is small
for x_val, coef in zip(x,y):
    if np.abs(coef)<0.05:
        ax.text(x_val, coef, '{:.3g}'.format(coef), rotation=45, va='bottom', ha='left')
ax.set_yticks(np.arange(-0.3,0.4,0.1))
ax.set_ylim(top=+0.3, bottom=-0.3)
# offset
x_val = x[0]
coef = y[0]
ax.text(x_val, 0.3, '{:.3g}'.format(coef), rotation=45, va='bottom', ha='left')

# plot origin
x_sign_switch = np.nonzero(np.array(y) < 0)[0][0]
ax.axvline(x_sign_switch-0.5, linewidth=2, linestyle='--', color='k')
ax.axhline(0, linewidth=2, linestyle='-', color='k')

# labels
ax.set_xticks(x)
ax.set_xticklabels(labels=x, rotation=45, va='top', ha='right', style='italic')
ax.xaxis.set_tick_params(which='minor', bottom=False)
ax.set_xlabel('Feature', style='italic')
ax.set_ylabel('Coefficient (a.u.)', style='italic')
ax.set_title('Multiple linear regression', style='italic') 

# stats
xy=[ax.get_xlim()[1], ax.get_ylim()[1]]
ax.text(xy[0]*1.05,xy[1], '$R^2$ = {:.2f}, $R^2_{{cv{:g}}}$ = {:.2f} (+/-{:.2f})'.format(
    models[model_name]['R^2'],
    models[model_name]['cv R^2'].shape[0],
    np.mean(models[model_name]['cv R^2']),
    np.std(models[model_name]['cv R^2']),
) + '\n' +
         'train (n = {})'.format(y_train.shape[0]) + '\n' +
         'test (n = {}, $R^2$ = {:.2f})'.format(
             y_test.shape[0],
             models[model_name]['test R^2'],
         ), style='italic', va='top', ha='left')


# Save
file_name = '../results/{}.png'.format(model_name.replace(' ','_'))
if (SKIPSAVE==False): #and (not(os.path.isfile(file_name))):
    print(file_name)
    with plt.style.context('../assets/context-paper.mplstyle'):
        plt.savefig(file_name, bbox_inches='tight', transparent=False)
else:
    plt.show()
    print(f'Skip. {file_name} exists or saving is disabled in settings.')

../results/MLR_impute_median.png


<a href="#pred_top" id='pred_model_6'><font size=+1><center>^^ TOP ^^</center></font></a>

---

# Model: MLR with categorical

As MLR, but do one-hot-encoding

Use different scalers for different columns:  
https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html  
https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html#sklearn.compose.ColumnTransformer  
p. 68 book: ML with sklearn & tf

## Prepare input

In [23]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import MinMaxScaler
# from sklearn.pipeline import FeatureUnion

model_name = 'MLR with categorical'

cat_columns_reduced = list(np.setdiff1d(cat_columns, ['model', 'fuel']))
features = num_columns + cat_columns_reduced
# Can be reduced here

# list of lists with categories. Needed for column transformer
cats = list(df[cat_columns_reduced].apply(lambda x:pd.Series(x.unique()).dropna().tolist() + ['missing'], axis='index'))

# Use data frame not array
yX = df.dropna(subset=['price'])
# # only use young
# is_yng = yX.age/365.25 < 25
# yX = yX[is_yng]
X = yX.iloc[:,1:]
y = yX.iloc[:,0]
print(X.shape)
print(y.shape)


(9019, 29)
(9019,)


In [24]:
import re

# Split fuel helper functions

def split_lpg_type(s):
    '''Split lpg type from list of fuels separated by / '''
    # No type
    if s.endswith('lpg'):
        return s, ''
    if 'lpg' not in s:
        return s, ''
    # Type is after the last '/'
    M = re.search('^(.*)/(.*)$',s)
    if M:
        return M[1], M[2]
    else:
        return s, ''

def merge_lpg_and_lpgtype(fuel_type):

    '''Add LPG type to LPG (remove /). 
    Note that order of fuels is preserved. I.e. it is able to return both "benzine/lpg-g3" and "lpg-g3/benzine". '''
    
    lpg_type = fuel_type.apply(lambda s: 'lpg-' + split_lpg_type(s)[1] if (type(s) == str) and ('lpg' in s) else '')
    fuel_type_short = fuel_type.apply(lambda s: split_lpg_type(s)[0] if (type(s) == str) else '')
    fuel_type_new = pd.Series([f.replace('lpg', l) if type(f) == str else f for f,l in zip(fuel_type_short,lpg_type)])
    return fuel_type_new


def get_unique_fuels(fuel_type):
    
    '''Splitting fuels at "/" and return unique values'''
    
    # make list (as string)
    fuel_type_list = fuel_type.apply(lambda s:s.split('/') if type(s) == str else np.NaN).astype(str)
    
    # Get unique fuels
    possible_fuels = list() # empty list
    for l in fuel_type_list.unique():
        for ll in eval(l): # use eval to convert str to list
            possible_fuels += [ll]     
    # uniquify
    return np.unique(possible_fuels)

    
from sklearn.base import BaseEstimator, TransformerMixin

# Custom transformer to make one-hot fuel encoder based on string
# This is different from get_dummies, because it can take a list of values in a field
class DummyfyFuel(BaseEstimator, TransformerMixin):
    def __init__(self, fuel_names=None):
        
        assert (fuel_names == None) or (isinstance(fuel_names, (list,))), '[fuel_names] should be list (or None)'
        
        self.fuel_names = fuel_names
        
    def fit(self, X, y=None):
        
        if not self.fuel_names:
            # get fuel names based on input.
            # Note that if train/test are split, test might lack a fuel type.
            self.fuel_names = get_unique_fuels(merge_lpg_and_lpgtype(X))

        return self
    
    def transform(self, X):
        
        # get stringyfied list
        fuel_type_list = merge_lpg_and_lpgtype(X).apply(lambda s:s.split('/') if type(s) == str else np.NaN).astype(str)
        # set index as input
        fuel_type_list.index = X.index

        # transform: dummies
        fuel_dummies = pd.DataFrame(index=fuel_type_list.index)
        for f in self.fuel_names:
            fuel_dummies['fuel_' + f] = fuel_type_list.apply(lambda l:int(f in eval(l)))

        return fuel_dummies


In [25]:
# instantiate a dict in models at key with name of this model
models[model_name] = dict()

# split train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)
print(X_train.shape)
print(X_test.shape)


(6313, 29)
(2706, 29)


In [26]:
# Create model

# Preprocessor: numerical features
num_transformer = make_pipeline(
    SimpleImputer(strategy='median'),
    MinMaxScaler(),
)
# Preprocessor: categorical features
cat_transformer = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='missing', missing_values=np.NaN),
    OneHotEncoder(categories=cats),
)

# Preprocess: fuels
# list of all fuels is passed by using full data set! (X)
fuel_list = list(get_unique_fuels(merge_lpg_and_lpgtype(X.fuel)))
#fuel_list = ['benzine', 'diesel']
get_fuel_dummies = DummyfyFuel(fuel_list)


# Combine num and cat
preprocessor = ColumnTransformer(transformers=[
    ('numerical', num_transformer, pd.Index(num_columns)),
    ('categorical', cat_transformer, pd.Index(cat_columns_reduced)),
    ('onehot_fuel', get_fuel_dummies, 'fuel')
], verbose=True)

# full pipeline with preproc and mlr
mlr = make_pipeline(
    preprocessor,
    linear_model.LinearRegression()
)

# Target transformation: log transform price
pl = TransformedTargetRegressor(
    regressor=mlr,
    func=np.log10,
    inverse_func=lambda y: 10**y,
#     func=lambda x:x,
#     inverse_func=lambda y: y,
#     inverse_func=np.exp,
)

models[model_name].update({'model':pl})

In [27]:
# fit
pl.fit(X_train, y_train)
y_pred = pl.predict(X_test)

[ColumnTransformer] ..... (1 of 3) Processing numerical, total=   0.0s
[ColumnTransformer] ... (2 of 3) Processing categorical, total=   0.0s
[ColumnTransformer] ... (3 of 3) Processing onehot_fuel, total=   0.4s


In [28]:
# sanity check that target transformation has occured as expected
# y_pred_manual_transform = mlr.predict(X_test)
# assert all(np.log10(y_pred)-y_pred_manual_transform == 0)

models[model_name].update({'n':y.shape[0]})
models[model_name].update({'n features':X.shape[1]})

# parameters
betas = [pl.regressor_.steps[-1][1].intercept_, *pl.regressor_.steps[-1][1].coef_]
models[model_name].update({'betas':betas})

# score
models[model_name].update({'R^2':pl.score(X,y)})
models[model_name].update({'test R^2':pl.score(X_test,y_test)})
cv_results = cross_val_score(pl, X_test, y_test, cv=5)
models[model_name].update({'cv R^2':cv_results})

[ColumnTransformer] ..... (1 of 3) Processing numerical, total=   0.0s
[ColumnTransformer] ... (2 of 3) Processing categorical, total=   0.0s
[ColumnTransformer] ... (3 of 3) Processing onehot_fuel, total=   0.1s
[ColumnTransformer] ..... (1 of 3) Processing numerical, total=   0.0s
[ColumnTransformer] ... (2 of 3) Processing categorical, total=   0.0s
[ColumnTransformer] ... (3 of 3) Processing onehot_fuel, total=   0.1s
[ColumnTransformer] ..... (1 of 3) Processing numerical, total=   0.0s
[ColumnTransformer] ... (2 of 3) Processing categorical, total=   0.0s
[ColumnTransformer] ... (3 of 3) Processing onehot_fuel, total=   0.1s
[ColumnTransformer] ..... (1 of 3) Processing numerical, total=   0.0s
[ColumnTransformer] ... (2 of 3) Processing categorical, total=   0.0s
[ColumnTransformer] ... (3 of 3) Processing onehot_fuel, total=   0.1s
[ColumnTransformer] ..... (1 of 3) Processing numerical, total=   0.0s
[ColumnTransformer] ... (2 of 3) Processing categorical, total=   0.0s
[Colum

In [29]:
# update features, by adding fuels
cat_columns_reduced += ['fuel']
cats += [fuel_list]


In [30]:
# Split betas per category feature.
idx_start = len(num_columns)+1
cat_betas = list()
for cat in cats:
    cat_betas += [betas[idx_start:idx_start+len(cat)]]
    idx_start += len(cat)
# Check if all betas are stored
assert cat_betas[0][0] == betas[len(num_columns)+1] # first cat beta follows numerical betas 
assert cat_betas[-1][-1] == betas[-1] # last

In [31]:
# plot coefficients

# plot numerical and catagorical in different subplots
n_plots = len(cat_columns_reduced) + 1
fig,axs=plt.subplots(
    nrows=n_plots,
    figsize=[16,4*n_plots]
)
plt.subplots_adjust(hspace=0.5)


# Plot numerical
ax = axs[0]
# sorted bar height
betas = models[model_name]['betas']
num_betas = betas[1:len(num_columns)+1]
x = ['offset'] + [features[i] for i in np.argsort(num_betas)[::-1]]
y = [betas[0]] + sorted(num_betas, reverse=True)

# plot bar
ax.bar(x=x, height=y, edgecolor='k', facecolor='None', clip_on=True)

# add values when bar is small
for x_val, coef in zip(x,y):
    if np.abs(coef)<0.5:
        ax.text(x_val, coef, '{:.3g}'.format(coef), rotation=45, va='bottom', ha='left')
ax.set_yticks(np.arange(-2,2.2,0.5))
ax.set_ylim(top=+2, bottom=-2)
# offset
x_val = x[0]
coef = y[0]
ax.text(x_val, 2, '{:.3g}'.format(coef), rotation=45, va='bottom', ha='left')

# plot origin
x_sign_switch = np.nonzero(np.array(y) < 0)[0][0]
ax.axvline(x_sign_switch-0.5, linewidth=2, linestyle='--', color='k')
ax.axhline(0, linewidth=2, linestyle='-', color='k')

# labels        
rot = 45
fsz = 10
ha = 'right'
ax.set_xticks(x)
ax.set_xticklabels(labels=x, rotation=rot, va='top', ha=ha, style='italic', fontsize=fsz)
ax.xaxis.set_tick_params(which='minor', bottom=False)
ax.set_ylabel('Coefficient (a.u.)', style='italic')
ax.set_title('Multiple linear regression\nNumerical features', style='italic') 

# stats
xy=[ax.get_xlim()[1], ax.get_ylim()[1]]
ax.text(xy[0]*1.05,xy[1], '$R^2$ = {:.2f}, $R^2_{{cv{:g}}}$ = {:.2f} (+/-{:.2f})'.format(
    models[model_name]['R^2'],
    models[model_name]['cv R^2'].shape[0],
    np.mean(models[model_name]['cv R^2']),
    np.std(models[model_name]['cv R^2']),
) + '\n' +
         'train (n = {})'.format(y_train.shape[0]) + '\n' +
         'test (n = {}, $R^2$ = {:.2f})'.format(
             y_test.shape[0],
             models[model_name]['test R^2'],
         ), style='italic', va='top', ha='left')

# Plot categorical
for cat, cat_beta, cat_name, ax in zip(cats, cat_betas, cat_columns_reduced, axs[1:]):
    # sort by height
    x = [cat[i] for i in np.argsort(cat_beta)[::-1]]
    y = sorted(cat_beta, reverse=True)
    #x = cat
    #y = cat_beta
    # plot bar
    ax.bar(x=x, height=y, edgecolor='k', facecolor='None', clip_on=False)

    # prettify
    ax.set_yticks(np.arange(-1,+1.1,0.2))
    ax.set_ylim(top=+1, bottom=-1)

    # plot origin
    x_sign_switch = np.nonzero(np.array(y) < 0)[0][0]
    ax.axvline(x_sign_switch-0.5, linewidth=2, linestyle='--', color='k')
    ax.axhline(0, linewidth=2, linestyle='-', color='k')

    # labels
    rot = 45
    fsz = 10
    ha = 'right'
    ax.set_xticks(x)
    ax.set_xticklabels(labels=x, rotation=rot, va='top', ha=ha, style='italic', fontsize=fsz)
    ax.xaxis.set_tick_params(which='minor', bottom=False)
    ax.set_title('Categorical feature: ' + cat_name, style='italic')
    ax.set_ylabel('Coefficient (a.u.)', style='italic')
    # add extra margin if bars are too wide (too little bars)
    if len(x) < 20:
        add_space = len(x) - 20
        xl = list(ax.set_xlim())
        xl[1] -= add_space/2
        xl[0] += add_space/2
        ax.set_xlim(xl)

# Label on bottom panel
ax = axs[-1]
ax.set_xlabel('Sorted features', style='italic')

# Save
file_name = '../results/{}.png'.format(model_name.replace(' ','_'))
if (SKIPSAVE==False): #and (not(os.path.isfile(file_name))):
    print(file_name)
    with plt.style.context('../assets/context-paper.mplstyle'):
        plt.savefig(file_name, bbox_inches='tight', transparent=False)
else:
    plt.show()
    print(f'Skip. {file_name} exists or saving is disabled in settings.')

../results/MLR_with_categorical.png


<a href="#pred_top" id='pred_model_7'><font size=+1><center>^^ TOP ^^</center></font></a>

---

# Model: MLR regularized

As [previous model](#pred_model_6), but use regularization by using built-in Lasso

## Prepare input

In [32]:
from sklearn.model_selection import GridSearchCV

model_name = 'MLR Lasso'

cat_columns_reduced = list(np.setdiff1d(cat_columns, ['model', 'fuel']))
features = num_columns + cat_columns_reduced
# Can be reduced here

# list of lists with categories. Needed for column transformer
cats = list(df[cat_columns_reduced].apply(lambda x:pd.Series(x.unique()).dropna().tolist() + ['missing'], axis='index'))

# Use data frame not array
yX = df.dropna(subset=['price'])
X = yX.iloc[:,1:]
y = yX.iloc[:,0]
print(X.shape)
print(y.shape)


(9019, 29)
(9019,)


## Determine regularization rate (alpha)

Alpha is the hyperparameter that needs to be determined. For this the data needs to be splitted, but the dataset is too small to do a 3 way split (i.e. CV, Train, Test). Therefor spilt 2 way k-fold cv 
- **Test**: Hold-out set for calculating performance
- **Train**: Use to fit model and do CV


In [33]:
# instantiate a dict in models at key with name of this model
models[model_name] = dict()

# split train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)
print(X_train.shape)
print(X_test.shape)


(6313, 29)
(2706, 29)


In [34]:
# Create model (same as MLR with cats, but regressor is Lasso)

# Preprocessor: numerical features
num_transformer = make_pipeline(
    SimpleImputer(strategy='median'),
    MinMaxScaler(),
)
# Preprocessor: categorical features
cat_transformer = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='missing', missing_values=np.NaN),
    OneHotEncoder(categories=cats),
)

# Preprocess: fuels
# list of all fuels is passed by using full data set! (X)
fuel_list = list(get_unique_fuels(merge_lpg_and_lpgtype(X.fuel)))
get_fuel_dummies = DummyfyFuel(fuel_list)


# Combine num and cat
preprocessor = ColumnTransformer(transformers=[
    ('numerical', num_transformer, pd.Index(num_columns)),
    ('categorical', cat_transformer, pd.Index(cat_columns_reduced)),
    ('onehot_fuel', get_fuel_dummies, 'fuel')
], verbose=False)

# full pipeline with preproc and mlr
mlr = make_pipeline(
    preprocessor,
    linear_model.Lasso(random_state=42, max_iter=2**11)
)

# Target transformation: log transform price
pl = TransformedTargetRegressor(
    regressor=mlr,
    func=np.log10,
    inverse_func=lambda y: 10**y
)



In [35]:
def gs_lasso_alpha(pipeline):
    # grid search estimator
    grid_search_alpha = GridSearchCV(
        estimator=pipeline,
        param_grid=[
            {
                'regressor__lasso__alpha': 10**(np.linspace(-5,-2,13)) # Choose alphas such that a clear peaked graph is shown in next plot
            } 
        ],
        cv=8,
        scoring='r2',
        #n_jobs=-1,
        verbose=10
    )

    # Perform grid search
    grid_search_alpha.fit(X_train,y_train);
    
    return grid_search_alpha

In [36]:
def plot_gscv_result(gscv, param_name = None):
    '''
    plot search results    
    '''
    
    if param_name is None:
        param_name = re.sub('^.+?__', '', list(gscv.param_grid[0].keys())[0])
    
    plt.figure(figsize=[2,2])

    # abscissa
    alphas = list(gscv.cv_results_['param_regressor__' + param_name])

    # plot mean
    r2_mean = gscv.cv_results_['mean_test_score']
    # normalize
    r2_mean = (r2_mean-r2_mean.mean())/r2_mean.std()
    plt.plot(alphas, r2_mean, label='mean', lw=4, color='blue')

    # plot folds
    for fold in range(gscv.cv):
        r2_fold = gscv.cv_results_['split{:.0f}_test_score'.format(fold)]
        # normalize
        r2_fold = (r2_fold-r2_fold.mean())/r2_fold.std()
        plt.plot(alphas, r2_fold, label='fold ' + str(fold), lw=1, color='black')

    plt.xscale('log')
    plt.xlabel('alpha')
    plt.ylabel('standardized r2 score [a.u.]')
    plt.axvline(gscv.best_params_['regressor__' + param_name], linewidth=2, linestyle='--', color='k')
    result = 'grid search results\nbest {:s}={:.5f}'.format(param_name, gscv.best_params_['regressor__' + param_name])
    plt.title(result)
    print(result)
    plt.legend(ncol=1, loc='center left', bbox_to_anchor=(1,0.5))
    
    return gscv.best_estimator_

### Fit with regressor found with grid search

In [37]:
best_estimator = plot_gscv_result(gs_lasso_alpha(pl))

Fitting 8 folds for each of 13 candidates, totalling 104 fits
[CV 1/8; 1/13] START regressor__lasso__alpha=1e-05..............................


/home/tom/bin/py310-satdatsci/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:609: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.0397734685328146, tolerance: 0.15405885004186756
  model = cd_fast.sparse_enet_coordinate_descent(


[CV 1/8; 1/13] END regressor__lasso__alpha=1e-05;, score=-0.941 total time=   3.1s
[CV 2/8; 1/13] START regressor__lasso__alpha=1e-05..............................


/home/tom/bin/py310-satdatsci/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:609: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.085811212882959, tolerance: 0.15485019750198498
  model = cd_fast.sparse_enet_coordinate_descent(


[CV 2/8; 1/13] END regressor__lasso__alpha=1e-05;, score=0.652 total time=   2.9s
[CV 3/8; 1/13] START regressor__lasso__alpha=1e-05..............................


...

[CV 6/8; 1/13] END regressor__lasso__alpha=1e-05;, score=0.379 total time=   2.9s
[CV 7/8; 1/13] START regressor__lasso__alpha=1e-05..............................


/home/tom/bin/py310-satdatsci/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:609: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.008853654918255, tolerance: 0.15402290993075862
  model = cd_fast.sparse_enet_coordinate_descent(


[CV 7/8; 1/13] END regressor__lasso__alpha=1e-05;, score=0.527 total time=   2.9s
[CV 8/8; 1/13] START regressor__lasso__alpha=1e-05..............................


/home/tom/bin/py310-satdatsci/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:609: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.0393994206548882, tolerance: 0.1531600141351679
  model = cd_fast.sparse_enet_coordinate_descent(


[CV 8/8; 1/13] END regressor__lasso__alpha=1e-05;, score=0.479 total time=   2.9s
[CV 1/8; 2/13] START regressor__lasso__alpha=1.778279410038923e-05..............
[CV 1/8; 2/13] END regressor__lasso__alpha=1.778279410038923e-05;, score=-0.933 total time=   2.0s
[CV 2/8; 2/13] START regressor__lasso__alpha=1.778279410038923e-05..............
[CV 2/8; 2/13] END regressor__lasso__alpha=1.778279410038923e-05;, score=0.662 total time=   2.0s
[CV 3/8; 2/13] START regressor__lasso__alpha=1.778279410038923e-05..............
[CV 3/8; 2/13] END regressor__lasso__alpha=1.778279410038923e-05;, score=0.453 total time=   2.1s
[CV 4/8; 2/13] START regressor__lasso__alpha=1.778279410038923e-05..............
[CV 4/8; 2/13] END regressor__lasso__alpha=1.778279410038923e-05;, score=0.492 total time=   2.7s
[CV 5/8; 2/13] START regressor__lasso__alpha=1.778279410038923e-05..............
[CV 5/8; 2/13] END regressor__lasso__alpha=1.778279410038923e-05;, score=0.832 total time=   2.1s
[CV 6/8; 2/13] START r

In [38]:
# Store estimator with best alpha
reg = best_estimator
models[model_name].update({'model':reg})

# fit
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

models[model_name].update({'n':y.shape[0]})
models[model_name].update({'n features':X.shape[1]})

# parameters
betas = [reg.regressor_.steps[-1][1].intercept_, *reg.regressor_.steps[-1][1].coef_]
models[model_name].update({'betas':betas})
models[model_name].update({'n betas effective':(np.abs(betas) > 0).sum()})

# score
models[model_name].update({'R^2':reg.score(X,y)})
models[model_name].update({'test R^2':reg.score(X_test,y_test)})
cv_results = cross_val_score(reg, X_test, y_test, cv=5)
models[model_name].update({'cv R^2':cv_results})

In [39]:
# update features, by adding fuels
cat_columns_reduced += ['fuel']
cats += [fuel_list]

# Split betas per category feature.
idx_start = len(num_columns)+1
cat_betas = list()
for cat in cats:
    cat_betas += [betas[idx_start:idx_start+len(cat)]]
    idx_start += len(cat)
# Check if all betas are stored
assert cat_betas[0][0] == betas[len(num_columns)+1] # first
assert cat_betas[-1][-1] == betas[-1] # last

In [40]:
# plot coefficients

# plot numerical and catagorical in different subplots
n_plots = len(cat_columns_reduced) + 1
fig,axs=plt.subplots(
    nrows=n_plots,
    figsize=[16,4*n_plots]
)
plt.subplots_adjust(hspace=0.5)

# Plot coefficients
for feats, coefs, name, ax in zip(
    [['offset'] + features] + cats,
    [[betas[0]] + betas[1:len(num_columns)+1]] + cat_betas,
    ['numerical'] + cat_columns_reduced,
    axs
):
    # sort by bar height
    x = [feats[i] for i in np.argsort(coefs)[::-1]]
    y = sorted(coefs, reverse=True)
    # plot bar
    ax.bar(x=x, height=y, edgecolor='k', facecolor='None', clip_on=True)

    # prettify
    if not name.startswith('num'):
        ax.set_yticks(np.arange(-0.5,+0.6,0.1))
        bot_tick, top_tick = ax.set_ylim(top=+0.5, bottom=-0.5)
    else:
        ax.set_yticks(np.arange(-2,2.2,0.5))
        bot_tick, top_tick = ax.set_ylim(top=+2, bottom=-2)
        # stats
        xy=[ax.get_xlim()[1], ax.get_ylim()[1]]
        ax.text(xy[0]*1.05,xy[1], '$R^2$ = {:.2f}, $R^2_{{cv{:g}}}$ = {:.2f} (+/-{:.2f})'.format(
            models[model_name]['R^2'],
            models[model_name]['cv R^2'].shape[0],
            np.mean(models[model_name]['cv R^2']),
            np.std(models[model_name]['cv R^2']),
        ) + '\n' +
                 'parameters total n={}, not zero n={}\n'.format(len(betas), sum(np.array(betas) != 0)) +
                 'train (n = {})'.format(y_train.shape[0]) + '\n' +
                 'test (n = {}, $R^2$ = {:.2f})'.format(
                     y_test.shape[0],
                     models[model_name]['test R^2'],
                 ), style='italic', va='top', ha='left')


    # plot sign switch
    x_sign_switch1 = np.nonzero(np.array(y+[-np.inf]) < 0)[0][0]
    x_sign_switch2 = np.nonzero(np.array([+np.inf]+y) > 0)[0][-1]
    ax.axvline(x_sign_switch1-0.5, linewidth=2, linestyle='--', color='k')
    ax.axvline(x_sign_switch2-0.5, linewidth=2, linestyle='--', color='k')
    ax.axhline(0, linewidth=2, linestyle='-', color='k')

    # add values when bar is small or too large (clipping)
    yt=ax.get_yticks()
    first_tick = sorted(np.abs(yt))[1]
    for x_val, coef in zip(x,y):
        if (coef < first_tick) & (coef > 0):
            ax.text(x_val, coef, '{:+.3g}'.format(coef), rotation=45, va='bottom', ha='left')
        elif (coef > -first_tick) & (coef < 0):
            ax.text(x_val, 0, '{:+.3g}'.format(coef), rotation=45, va='bottom', ha='left')
        elif coef > top_tick:
            # generally this is offset (bias)
            ax.text(x_val, top_tick, '{:+.3g}'.format(coef), rotation=45, va='bottom', ha='left')
        elif coef < bot_tick:
            ax.text(x_val, bot_tick, '{:+.3g}'.format(coef), rotation=45, va='bottom', ha='left')

    
    # labels and titles
    rot = 45
    fsz = 10
    ha = 'right'
    ax.set_xticks(x)
    ax.set_xticklabels(labels=x, rotation=rot, va='top', ha=ha, style='italic', fontsize=fsz)
    ax.xaxis.set_tick_params(which='minor', bottom=False)
    if not name.startswith('num'):
        ax.set_title('Categorical feature: ' + name, style='italic')
    else:
        ax.set_title('Multiple linear regression (Lasso, alpha={:g})\nNumerical features'.format(
            reg.regressor_.named_steps['lasso'].alpha
        ), style='italic') 
    ax.set_ylabel('Coefficient (a.u.)', style='italic')
    
    # add extra margin if bars are too wide (too little bars)
    if len(x) < 20:
        add_space = len(x) - 20
        xl = list(ax.get_xlim())
        xl[1] -= add_space/2
        xl[0] += add_space/2
        ax.set_xlim(xl)

# Label on bottom panel
ax = axs[-1]
ax.set_xlabel('Sorted features', style='italic')

# Save
file_name = '../results/{}.png'.format(model_name.replace(' ','_'))
if (SKIPSAVE==False): #and (not(os.path.isfile(file_name))):
    print(file_name)
    with plt.style.context('../assets/context-paper.mplstyle'):
        plt.savefig(file_name, bbox_inches='tight', transparent=False)
else:
    plt.show()
    print(f'Skip. {file_name} exists or saving is disabled in settings.')

../results/MLR_Lasso.png


<a href="#pred_top" id='pred_model_8'><font size=+1><center>^^ TOP ^^</center></font></a>

---

# Model: MLR regularized with added feature

As previous, but with odometer and age combine into one feature

In [41]:
# check to see if combining features would improve model
yX = df.dropna(subset=['price']).copy()
yX.loc[:,'> usage_intensity <'] = (yX.odometer / yX.age)
yX.loc[:,'> is_classic <'] = yX.age > 25
print(yX.corr().price.sort_values())
print('\n"usage_intensity" does not seem to correlate better than "age" and "odometer" seperately')

odometer                        -0.438026
age                             -0.386446
private_owners                  -0.277994
days_since_inspection_invalid   -0.108336
converted_energy_label          -0.091929
> usage_intensity <             -0.049017
number_of_seats                 -0.025353
> is_classic <                  -0.021214
age_at_import                    0.045815
height                           0.070208
company_owners                   0.115647
number_of_doors                  0.178647
length                           0.300840
number_of_cylinders              0.358457
weight                           0.368128
registration_tax                 0.372519
displacement                     0.380830
width                            0.444096
top_speed                        0.509616
number_of_gears                  0.571781
power                            0.613989
original_sale_price              0.691191
price                            1.000000
Name: price, dtype: float64

"usag

## Prepare input

In [42]:
from sklearn.model_selection import GridSearchCV

model_name = 'MLR added features'

cat_columns_reduced = list(np.setdiff1d(cat_columns, ['model', 'fuel']))
# Can be reduced here

# list of lists with categories. Needed for column transformer
cats = list(df[cat_columns_reduced].apply(lambda x:pd.Series(x.unique()).dropna().tolist() + ['missing'], axis='index'))

# Use data frame not array
yX = df.dropna(subset=['price'])
X = yX.iloc[:,1:]
y = yX.iloc[:,0]

# Add features
X.loc[:,'usage_intensity'] = X.odometer / X.age
X.loc[:,'classic'] = X.age > 25*365
X.loc[:,'classic'].replace({True:'y', False:'n'}, inplace=True)
#X.loc[X.age.isna(),'classic'] = 'missing' # is done in imputer

print(X.shape)
print(y.shape)

(9019, 31)
(9019,)


In [43]:
cat_columns_expanded = cat_columns_reduced + ['classic']
cats_added = cats + [['y', 'n', 'missing']]
num_columns_expanded = num_columns + ['usage_intensity']
# num_columns_expanded.remove('age')
num_columns_expanded.remove('odometer')
features = num_columns_expanded + cat_columns_expanded


In [44]:
# instantiate a dict in models at key with name of this model
models[model_name] = dict()

# split train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)
print(X_train.shape)
print(X_test.shape)


(6313, 31)
(2706, 31)


In [45]:
# Create model (same as MLR Lasso)

# Preprocessor: numerical features
num_transformer = make_pipeline(
    SimpleImputer(strategy='median'),
    MinMaxScaler(),
)
# Preprocessor: categorical features
cat_transformer = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='missing', missing_values=np.NaN),
    OneHotEncoder(categories=cats_added),
)

# Preprocess: fuels
# list of all fuels is passed by using full data set! (X)
fuel_list = list(get_unique_fuels(merge_lpg_and_lpgtype(X.fuel)))
get_fuel_dummies = DummyfyFuel(fuel_list)


# Combine num and cat
preprocessor = ColumnTransformer(transformers=[
    ('numerical', num_transformer, pd.Index(num_columns_expanded)),
    ('categorical', cat_transformer, pd.Index(cat_columns_expanded)),
    ('onehot_fuel', get_fuel_dummies, 'fuel')
], verbose=False)

# full pipeline with preproc and mlr
mlr = make_pipeline(
    preprocessor,
    linear_model.Lasso(random_state=42)
)

# Target transformation: log transform price
pl = TransformedTargetRegressor(
    regressor=mlr,
    func=np.log10,
    inverse_func=lambda y: 10**y
)



In [46]:
# # use alpha from previous model
# alpha = models['MLR Lasso']['model'].regressor['lasso'].get_params()['alpha']
# pl.regressor['lasso'].set_params(alpha = alpha)

best_estimator = plot_gscv_result(gs_lasso_alpha(pl), 'lasso__alpha')



Fitting 8 folds for each of 13 candidates, totalling 104 fits
[CV 1/8; 1/13] START regressor__lasso__alpha=1e-05..............................


/home/tom/bin/py310-satdatsci/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:609: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 17.828293802722044, tolerance: 0.15405885004186756
  model = cd_fast.sparse_enet_coordinate_descent(


[CV 1/8; 1/13] END regressor__lasso__alpha=1e-05;, score=-2.258 total time=   1.7s
[CV 2/8; 1/13] START regressor__lasso__alpha=1e-05..............................


/home/tom/bin/py310-satdatsci/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:609: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 42.852810441242525, tolerance: 0.15485019750198498
  model = cd_fast.sparse_enet_coordinate_descent(


[CV 2/8; 1/13] END regressor__lasso__alpha=1e-05;, score=0.615 total time=   1.8s
[CV 3/8; 1/13] START regressor__lasso__alpha=1e-05..............................


...

[CV 5/8; 5/13] END regressor__lasso__alpha=0.0001;, score=0.395 total time=   1.6s
[CV 6/8; 5/13] START regressor__lasso__alpha=0.0001.............................
[CV 6/8; 5/13] END regressor__lasso__alpha=0.0001;, score=0.482 total time=   0.7s
[CV 7/8; 5/13] START regressor__lasso__alpha=0.0001.............................


/home/tom/bin/py310-satdatsci/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:609: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.1567362587527441, tolerance: 0.15402290993075862
  model = cd_fast.sparse_enet_coordinate_descent(


[CV 7/8; 5/13] END regressor__lasso__alpha=0.0001;, score=0.441 total time=   1.5s
[CV 8/8; 5/13] START regressor__lasso__alpha=0.0001.............................


/home/tom/bin/py310-satdatsci/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:609: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.23011148299815432, tolerance: 0.1531600141351679
  model = cd_fast.sparse_enet_coordinate_descent(


[CV 8/8; 5/13] END regressor__lasso__alpha=0.0001;, score=0.457 total time=   1.5s
[CV 1/8; 6/13] START regressor__lasso__alpha=0.00017782794100389227.............
[CV 1/8; 6/13] END regressor__lasso__alpha=0.00017782794100389227;, score=0.108 total time=   0.9s
[CV 2/8; 6/13] START regressor__lasso__alpha=0.00017782794100389227.............
[CV 2/8; 6/13] END regressor__lasso__alpha=0.00017782794100389227;, score=0.628 total time=   1.1s
[CV 3/8; 6/13] START regressor__lasso__alpha=0.00017782794100389227.............
[CV 3/8; 6/13] END regressor__lasso__alpha=0.00017782794100389227;, score=0.446 total time=   1.1s
[CV 4/8; 6/13] START regressor__lasso__alpha=0.00017782794100389227.............
[CV 4/8; 6/13] END regressor__lasso__alpha=0.00017782794100389227;, score=0.433 total time=   0.8s
[CV 5/8; 6/13] START regressor__lasso__alpha=0.00017782794100389227.............
[CV 5/8; 6/13] END regressor__lasso__alpha=0.00017782794100389227;, score=0.301 total time=   0.9s
[CV 6/8; 6/13] ST

### Fit

In [47]:
# Store estimator
reg = best_estimator
models[model_name].update({'model':reg})

# fit
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

models[model_name].update({'n':y.shape[0]})
models[model_name].update({'n features':X.shape[1]})

# parameters
betas = [reg.regressor_.steps[-1][1].intercept_, *reg.regressor_.steps[-1][1].coef_]
models[model_name].update({'betas':betas})
models[model_name].update({'n betas effective':(np.abs(betas) > 0).sum()})

# score
models[model_name].update({'R^2':reg.score(X,y)})
models[model_name].update({'test R^2':reg.score(X_test,y_test)})
cv_results = cross_val_score(reg, X_test, y_test, cv=5)
models[model_name].update({'cv R^2':cv_results})

In [48]:
# update features, by adding fuels
cat_columns_expanded += ['fuel']
cats_added += [fuel_list]


In [49]:
# Split betas per category feature.
idx_start = len(num_columns_expanded)+1
cat_betas = list()
for cat in cats_added:
    cat_betas += [betas[idx_start:idx_start+len(cat)]]
    idx_start += len(cat)
# Check if all betas are stored
assert cat_betas[0][0] == betas[len(num_columns_expanded)+1] # first
assert cat_betas[-1][-1] == betas[-1] # last

In [50]:
# plot coefficients

# plot numerical and catagorical in different subplots
n_plots = len(cat_columns_expanded) + 1
fig,axs=plt.subplots(
    nrows=n_plots,
    figsize=[16,4*n_plots]
)
plt.subplots_adjust(hspace=0.5)

# Plot coefficients
for feats, coefs, name, ax in zip(
    [['offset'] + features] + cats_added,
    [[betas[0]] + betas[1:len(num_columns_expanded)+1]] + cat_betas,
    ['numerical'] + cat_columns_expanded,
    axs
):
    # sort by bar height
    x = [feats[i] for i in np.argsort(coefs)[::-1]]
    y = sorted(coefs, reverse=True)
    # plot bar
    ax.bar(x=x, height=y, edgecolor='k', facecolor='None', clip_on=True)

    # prettify
    if not name.startswith('num'):
        ax.set_yticks(np.arange(-0.5,+0.6,0.1))
        bot_tick, top_tick = ax.set_ylim(top=+0.5, bottom=-0.5)
    else:
        ax.set_yticks(np.arange(-2,2.2,0.5))
        bot_tick, top_tick = ax.set_ylim(top=+2, bottom=-2)
        # stats
        xy=[ax.get_xlim()[1], ax.get_ylim()[1]]
        ax.text(xy[0]*1.05,xy[1], '$R^2$ = {:.2f}, $R^2_{{cv{:g}}}$ = {:.2f} (+/-{:.2f})'.format(
            models[model_name]['R^2'],
            models[model_name]['cv R^2'].shape[0],
            np.mean(models[model_name]['cv R^2']),
            np.std(models[model_name]['cv R^2']),
        ) + '\n' +
                 'parameters total n={}, not zero n={}\n'.format(len(betas), sum(np.array(betas) != 0)) +
                 'train (n = {})'.format(y_train.shape[0]) + '\n' +
                 'test (n = {}, $R^2$ = {:.2f})'.format(
                     y_test.shape[0],
                     models[model_name]['test R^2'],
                 ), style='italic', va='top', ha='left')


    # plot sign switch
    x_sign_switch1 = np.nonzero(np.array(y+[-np.inf]) < 0)[0][0]
    x_sign_switch2 = np.nonzero(np.array([+np.inf]+y) > 0)[0][-1]
    ax.axvline(x_sign_switch1-0.5, linewidth=2, linestyle='--', color='k')
    ax.axvline(x_sign_switch2-0.5, linewidth=2, linestyle='--', color='k')
    ax.axhline(0, linewidth=2, linestyle='-', color='k')

    # add values when bar is small or too large (clipping)
    yt=ax.get_yticks()
    first_tick = sorted(np.abs(yt))[1]
    for x_val, coef in zip(x,y):
        if (coef < first_tick) & (coef > 0):
            ax.text(x_val, coef, '{:+.3g}'.format(coef), rotation=45, va='bottom', ha='left')
        elif (coef > -first_tick) & (coef < 0):
            ax.text(x_val, 0, '{:+.3g}'.format(coef), rotation=45, va='bottom', ha='left')
        elif coef > top_tick:
            # generally this is offset (bias)
            ax.text(x_val, top_tick, '{:+.3g}'.format(coef), rotation=45, va='bottom', ha='left')
        elif coef < bot_tick:
            ax.text(x_val, bot_tick, '{:+.3g}'.format(coef), rotation=45, va='bottom', ha='left')

    
    # labels and titles
    rot = 45
    fsz = 10
    ha = 'right'
    ax.set_xticks(x)
    ax.set_xticklabels(labels=x, rotation=rot, va='top', ha=ha, style='italic', fontsize=fsz)
    ax.xaxis.set_tick_params(which='minor', bottom=False)
    if not name.startswith('num'):
        ax.set_title('Categorical feature: ' + name, style='italic')
    else:
        ax.set_title('Multiple linear regression (Lasso, alpha={:g})\nNumerical features'.format(
            reg.regressor_.named_steps['lasso'].alpha
        ), style='italic') 
    ax.set_ylabel('Coefficient (a.u.)', style='italic')
    
    # add extra margin if bars are too wide (too little bars)
    if len(x) < 10:
        add_space = len(x) - 20
        xl = list(ax.get_xlim())
        xl[1] -= add_space/2
        xl[0] += add_space/2
        ax.set_xlim(xl)

# Label on bottom panel
ax = axs[-1]
ax.set_xlabel('Sorted features', style='italic')

# Save
file_name = '../results/{}.png'.format(model_name.replace(' ','_'))
if (SKIPSAVE==False): #and (not(os.path.isfile(file_name))):
    print(file_name)
    with plt.style.context('../assets/context-paper.mplstyle'):
        plt.savefig(file_name, bbox_inches='tight', transparent=False)
else:
    plt.show()
    print(f'Skip. {file_name} exists or saving is disabled in settings.')

../results/MLR_added_features.png


- - - - - 

In [51]:
# Display prediction errors

x_sample = df.dropna(subset=['price']).iloc[:,1:]
y_sample = df.dropna(subset=['price']).iloc[:,0]
# Add features
x_sample.loc[:,'usage_intensity'] = x_sample.odometer / x_sample.age
x_sample.loc[:,'classic'] = x_sample.age > 25*365
x_sample.loc[:,'classic'].replace({True:'y', False:'n'}, inplace=True)
# predict again
y_sample_pred = models[model_name]['model'].predict(x_sample) 

x_sample['price'] = y_sample
x_sample['prediction_error'] = y_sample_pred - y_sample
x_sample['prediction_error_fraction'] = y_sample_pred/y_sample
x_sample['prediction_error_log'] = np.log10(x_sample.prediction_error_fraction)
x_sample['prediction_error_abslog'] = np.abs(np.log10(x_sample.prediction_error_fraction))
x_sample['prediction'] = y_sample_pred
x_sample['age_y'] = x_sample.age/365

# Note some are close to perfect, because they are in training set and are unique in brand etc
print('best predictons')
display(x_sample.sort_values(by='prediction_error_abslog').head(16).T)
print('worst predictions')
display(x_sample.sort_values(by='prediction_error_abslog').tail(16).T)
print('largest underestimate')
display(x_sample.sort_values(by='prediction_error').head(16).T)
print('largest overestimate')
display(x_sample.sort_values(by='prediction_error').tail(16).T)
print('worst prediction recent auction')
is_last_auction = x_sample.index.str.startswith('-'.join(x_sample.index[-1].split('-')[:2]))
display(x_sample[is_last_auction].sort_values(by='prediction_error_abslog').tail(8).T)

plt.figure(figsize=[8,8])
plt.plot(x_sample.age_y, x_sample.prediction_error_log, color='k', marker='s', markeredgecolor = (0, 0, 0, 0), markerfacecolor = (0, 0, 0, 1), linestyle='None', ms=4)
plt.axhline(0, lw=2, linestyle='--', color ='k')
plt.xlabel('age [years]')
plt.ylabel('prediction error [log of fraction]\n(positive: prediction overestimates)')
plt.show()

best predictons


,2022-03-705903,2022-01-700301,2022-07-702207,2022-01-802321,2019-10-7121,2017-11-2206,2023-03-708703,2022-09-703309,2019-1-8205,2018-11-8125,2023-03-703803,2021-05-8116,2017-11-2223,2017-11-2613,2022-11-706611,2022-07-709107
brand,BMW,AUDI,VOLKSWAGEN,VOLKSWAGEN,PEUGEOT,VOLVO,MERCEDES-BENZ,RENAULT,VOLVO,BMW,VOLKSWAGEN,VOLKSWAGEN,DODGE,VOLKSWAGEN,VOLKSWAGEN,VOLKSWAGEN
model,3-serie touring,a3 sportback,polo,polo,207,v40,a 170,megane,v50,1er reihe,golf,touran,journey,polo,polo,golf
age,NaN,2556.0,3892.0,2038.0,4654.0,1134.0,5642.0,3639.0,2542.0,4902.0,4662.0,NaN,3191.0,2540.0,3941.0,4921.0
fuel,Diesel,Diesel,Diesel,Benzine,Diesel,Diesel,Benzine,Diesel,Diesel,Diesel,Diesel,Diesel,Diesel,Benzine,Diesel,Benzine
odometer,95254.0,287871.0,241983.0,167084.0,188230.0,138527.0,241288.0,190969.0,196715.0,308196.0,257721.0,22203.0,253318.0,64576.0,207602.0,75185.0
days_since_inspection_invalid,NaN,-137.0,-126.0,-236.0,3558.0,-651.0,-13.0,352.0,-38.0,-2.0,252.0,NaN,-535.0,-742.0,709.0,71.0
age_at_import,NaN,0.0,0.0,0.0,4389.0,1007.0,2704.0,0.0,0.0,0.0,0.0,NaN,0.0,2552.0,1367.0,1467.0
body_type,NaN,Hatchback,Hatchback,Hatchback,Hatchback,MPV,Stationwagen,Stationwagen,MPV,Hatchback,Stationwagen,NaN,MPV,Hatchback,Hatchback,Stationwagen
displacement,NaN,1598.0,1199.0,999.0,1560.0,1560.0,1699.0,1461.0,1560.0,1995.0,1598.0,NaN,1968.0,1198.0,1598.0,1390.0
number_of_cylinders,NaN,4.0,3.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,NaN,4.0,3.0,4.0,4.0


worst predictions


,2022-12-260032,2017-6-2200,2022-05-260625,2018-8-2410,2020-9-3035,2017-8-7139,2021-09-260619,2019-4-2021,2018-8-2400,2022-09-265029,2015-02-2200,2015-03-2402,2018-7-2415,2017-5-2216,2014-12-2207,2017-3-2000
brand,CHEVROLET,CHEVROLET,FERRARI,ASTON-MARTIN,PEUGEOT,BMW,LOTUS,VOLKSWAGEN,ROLLS ROYCE,LAMBORGHINI,ASTON-MARTIN,FORD,AUSTIN-HEALEY,ALFA ROMEO,FORD,ALFA ROMEO
model,impala sport coupe,camaro sport coupe,430 scuderia,dbs,306,5er reihe,elise (lhd & rhd),111011,phantom drophead coupe,urus,vanguish volante,thunderbird,3000 mkiii phase ii,2000 gtv,thunderbird,2000 gtv
age,20000.0,16773.0,NaN,NaN,NaN,NaN,8377.0,19787.0,NaN,NaN,NaN,21063.0,19116.0,16257.0,20973.0,16196.0
fuel,Benzine,Benzine,Benzine,Benzine,Diesel,Benzine,Benzine,Benzine,Benzine,Benzine,Benzine,Benzine,Benzine,Benzine,Benzine,Benzine
odometer,73172.043648,9137.0,11077.0,58429.0,NaN,362604.0,87992.0,91157.0,11305.0,17340.0,4778.0,86207.0,106800.895872,23982.0,86207.0,23982.0
days_since_inspection_invalid,NaN,-278.0,NaN,NaN,NaN,NaN,-288.0,1099.0,NaN,NaN,NaN,NaN,-402.0,-739.0,NaN,-800.0
age_at_import,19118.0,1261.0,NaN,NaN,NaN,NaN,0.0,16180.0,NaN,NaN,NaN,12826.0,18063.0,0.0,12826.0,0.0
body_type,Coupe,NaN,NaN,NaN,NaN,NaN,Cabriolet,Sedan,NaN,NaN,NaN,NaN,Cabriolet,Coupe,NaN,Coupe
displacement,5358.0,NaN,NaN,NaN,NaN,NaN,1796.0,1192.0,NaN,NaN,NaN,NaN,2912.0,NaN,NaN,NaN
number_of_cylinders,8.0,8.0,NaN,NaN,NaN,NaN,4.0,4.0,NaN,NaN,NaN,8.0,6.0,4.0,8.0,4.0


largest underestimate


,2022-09-265029,2023-04-200304,2015-02-2200,2021-12-260012,2018-8-2400,2022-05-260625,2019-11-2418,2019-4-2411,2015-03-2420,2015-01-2414,2018-6-2410,2023-04-260104,2022-05-260925,2021-05-8126,2018-8-2410,2014-12-2221
brand,LAMBORGHINI,PORSCHE,ASTON-MARTIN,LAMBORGHINI,ROLLS ROYCE,FERRARI,PORSCHE,MERCEDES-BENZ,BENTLEY,SKODA,MERCEDES-BENZ,LAND ROVER,LAND ROVER,MERCEDES-BENZ,ASTON-MARTIN,BENTLEY
model,urus,911 turbo s,vanguish volante,132 se,phantom drophead coupe,430 scuderia,panamera turbo s e-hybrid,amg s63 cabriolet,contintal gt 60w12 gtc,octavia,S65 AMG,autobiography d350,range rover 3.0 lwb autobiogra,v-klasse,dbs,continental gtc
age,NaN,NaN,NaN,8546.0,NaN,NaN,NaN,NaN,NaN,315.0,NaN,NaN,NaN,1506.0,NaN,NaN
fuel,Benzine,Benzine,Benzine,Benzine,Benzine,Benzine,Hybrid,Benzine,Benzine,Benzine,Benzine,Diesel,Benzine,Diesel,Benzine,Benzine
odometer,17340.0,1285.0,4778.0,40090.0,11305.0,11077.0,6925.0,13.0,18210.0,6796.0,6379.0,734.0,183.0,121324.0,58429.0,67890.0
days_since_inspection_invalid,NaN,NaN,NaN,1949.0,NaN,NaN,NaN,NaN,NaN,-2972.0,NaN,NaN,NaN,-320.0,NaN,NaN
age_at_import,NaN,NaN,NaN,6164.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1067.0,NaN,NaN
body_type,NaN,NaN,NaN,Coupe,NaN,NaN,NaN,NaN,NaN,Sedan,NaN,NaN,NaN,MPV,NaN,NaN
displacement,NaN,NaN,NaN,5707.0,NaN,NaN,NaN,NaN,NaN,1798.0,NaN,NaN,NaN,2143.0,NaN,NaN
number_of_cylinders,NaN,NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,4.0,NaN,NaN


largest overestimate


,2023-08-707708,2022-09-701509,2014-11-2601,2017-3-2405,2021-03-2206,2020-1-2414,2022-09-704409,2019-12-2407,2023-10-262310,2023-09-261909,2022-11-705411,2015-02-2202,2023-04-700604,2022-11-260311,2021-06-702006,2021-03-2610
brand,MERCEDES-BENZ,LAND ROVER,MERCEDES-BENZ,MERCEDES-BENZ,VOLVO,MERCEDES-BENZ,BMW,MERCEDES-BENZ,AUDI,PORSCHE,BMW,PORSCHE,BMW,BMW,PORSCHE,LAND ROVER
model,amg gle 53 4matic+ coupe,range rover evoque,sl 350,amg gle 63 s,xc90 t8 twin engine,amg gle 63 s,x6 m,amg c 63 s,rs q8,panamera s e-hybrid,x5 m,panamera 4s,x5 m,z4 m40i,macan s,range rover sport
age,1175.0,625.0,869.0,484.0,579.0,1063.0,1695.0,1059.0,753.0,3140.0,2569.0,1602.0,2720.0,733.0,783.0,1851.0
fuel,Benzine/Elektriciteit,Benzine/Elektriciteit,Benzine,Benzine,Elektriciteit/Benzine,Benzine,Benzine,Benzine,Benzine/Elektriciteit,Benzine/Elektriciteit,Benzine,Benzine,Benzine,Benzine,Benzine,Benzine
odometer,18353.0,358.0,10180.0,20757.0,7071.0,31404.0,119198.0,22356.0,19510.0,119785.0,73626.0,43374.0,73626.0,8197.0,25070.0,110979.0
days_since_inspection_invalid,-286.0,-836.0,-2785.0,-977.0,-882.0,-398.0,234.0,-402.0,-708.0,218.0,-432.0,-2569.0,-281.0,-728.0,-678.0,-121.0
age_at_import,575.0,0.0,588.0,135.0,0.0,526.0,309.0,329.0,255.0,0.0,2270.0,645.0,2270.0,0.0,0.0,1246.0
body_type,Stationwagen,Stationwagen,Cabriolet,Stationwagen,MPV,Stationwagen,Sedan,Cabriolet,Hatchback,Hatchback,Stationwagen,Hatchback,Stationwagen,Cabriolet,Stationwagen,Stationwagen
displacement,2999.0,1997.0,3498.0,5461.0,1969.0,5461.0,4395.0,3982.0,3996.0,2995.0,4395.0,4806.0,4395.0,2998.0,2995.0,4999.0
number_of_cylinders,6.0,4.0,6.0,8.0,4.0,8.0,8.0,8.0,8.0,6.0,8.0,8.0,8.0,6.0,6.0,8.0


worst prediction recent auction


,2023-10-700510,2023-10-705610,2023-10-705310,2023-10-701210,2023-10-705510,2023-10-705110,2023-10-262210,2023-10-700710
brand,OPEL,ALFA ROMEO,BMW,MITSUBISHI,FORD,BMW,FERRARI,MAZDA
model,zafira,alfa giulietta,530 xd,outlander,focus,5er reihe,430,6
age,NaN,4783.0,NaN,4132.0,6438.0,NaN,6331.0,NaN
fuel,Diesel,Benzine,Diesel,Benzine,Benzine,Diesel,Benzine,Benzine
odometer,163101.0,151976.0,294157.0,230792.0,144807.0,417983.0,118107.0,284874.0
days_since_inspection_invalid,NaN,371.0,NaN,261.0,-128.0,NaN,124.0,NaN
age_at_import,NaN,0.0,NaN,0.0,0.0,NaN,2897.0,NaN
body_type,NaN,MPV,NaN,Stationwagen,Hatchback,NaN,Coupe,NaN
displacement,NaN,1742.0,NaN,1998.0,2521.0,NaN,4308.0,NaN
number_of_cylinders,NaN,4.0,NaN,4.0,5.0,NaN,8.0,NaN


<a href="#pred_top" id='pred_accuracies'><font size=+1><center>^^ TOP ^^</center></font></a>

---

## Model accuracies

In [52]:
# plot R^2

# counter for x-offset
c=0

# figure
fig = plt.figure(figsize=[4,2])
ax = fig.gca()
xs = ys = fs = [None]

# loop over all models
for name,res in models.items():

    c+=1 # x-offset

    if name == 'linear regression no cv':
        # No cv, so only one value. Make it a list of one for type consistency
        k = 'R^2'
        rsq = [res[k]]
    
    else: 
        k = 'cv R^2'
        rsq = res[k]
        
    if 'n betas effective' in res:
        ndf = res['n betas effective']
    else:
        ndf = len(res['betas'])
        
    # add r-squares and offset to vectors
    ys = np.concatenate([ys, rsq])
    xs = np.concatenate([xs, np.ones_like(rsq) * c])
    fs = np.concatenate([fs, [ndf]])

# actual plotting
sns.swarmplot(x=xs, y=ys, ax=ax, hue=None)
ax.bar(range(1,len(models)+1), [res['R^2'] for res in models.values()], width=0.8, fc='none')
for x,ndf in enumerate(fs):
    if ndf is None:
        continue
    if x == 1:
        s = f'd.f.: {ndf:.0f}'
    else:
        s = f'{ndf:.0f}'
    ax.text(x, 1, s, ha='center')
# prettify
ax.set_xticks(range(1,len(models)+1))
ax.set_xticklabels(labels=list(models.keys()), rotation=45, va='top', ha='right', style='italic')
ax.set_ylim(bottom=0, top=+1)
ax.set_title('Model performance\n', style='italic')
ax.set_ylabel('Coefficient of determination\n($R^2$)', style='italic')
ax.xaxis.set_tick_params(which='minor', bottom=False)

# save
file_name = '../results/model-performance.png'
if (SKIPSAVE==False): #and (not(os.path.isfile(file_name))):
    print(file_name)
    with plt.style.context('../assets/context-paper.mplstyle'):
        plt.savefig(file_name, bbox_inches='tight', transparent=False)
else:
    plt.show()
    print(f'Skip. {file_name} exists or saving is disabled in settings.')

../results/model-performance.png


In [53]:
# plot data

# loop over all models
for model_name in models.keys():
    print(model_name)
    res = models[model_name]
    
    # all original data
    yX = df.loc[:,['price', 'age']].dropna()
    X = yX.iloc[:,1]
    y = yX.iloc[:,0]
    
    features = num_columns.copy()
    
    # model specific adjustments
    if (model_name == 'linear regression log price') or (model_name == 'linear regression log price young'):
        # log price is used
        y = np.log10(y)
        # unit
        unit = '(log[EUR])'
    elif (model_name == 'MLR reduced observations') or (model_name == 'MLR impute median'):
        yX = df.dropna(subset=['price'] + features).loc[:,['price'] + features]
        X = yX.iloc[:,1:]
        y = np.log10(yX.iloc[:,0])
        unit = '(log[EUR])'
    elif (model_name == 'MLR with categorical') or (model_name == 'MLR Lasso'):
        yX = df.dropna(subset=['price']).copy()
        X = yX.iloc[:,1:]
        y = yX.iloc[:,0]      
        unit = '(EUR)'
    elif (model_name == 'MLR added features'):
        yX = df.dropna(subset=['price']).copy()
        X = yX.iloc[:,1:]
        y = yX.iloc[:,0]      
        unit = '(EUR)'
        X.loc[:,'usage_intensity'] = X.odometer / X.age
        X.loc[:,'classic'] = X.age > 25*365
        X.loc[:,'classic'].replace({True:'y', False:'n'}, inplace=True)
    elif (model_name == 'Decision Tree Regression'):
        yX = df.dropna(subset=['price']).copy()
        X = yX.iloc[:,1:]
        y = yX.iloc[:,0]      
        unit = '(EUR)'
        X.loc[:,'usage_intensity'] = X.odometer / X.age
        X.loc[:,'classic'] = X.age > 25*365
        X.loc[:,'classic'].replace({True:'y', False:'n'}, inplace=True)

    else:
        unit = '(EUR)'
    
    if X.ndim != 1:
        n_feat = X.shape[1]
    else:
        n_feat = 1
        
    if not ((model_name == 'MLR with categorical') or (model_name == 'MLR Lasso') or (model_name == 'MLR added features') or (model_name == 'Decision Tree Regression')):
        # needed for .predict
        X = np.array(X).reshape(-1,n_feat)
        y = np.array(y).reshape(-1,1)
    
    # predict all data
    y_pred = res['model'].predict(X)
    if max(y) < 10:
        rmse = np.sqrt(np.mean(((10**y)-(10**y_pred))**2))
    else:
        rmse = np.sqrt(np.mean((y-y_pred)**2))
    print(rmse)

    # actual plotting
    fig,axs = plt.subplots(nrows=2, ncols=1, figsize=[8,8])
    
    # data
    #axs[0].plot(y, y_pred, marker='s', markeredgecolor = (0, 0, 0, 1), markerfacecolor = (1, 1, 1, .5), linestyle='None', ms=4)
    axs[0].plot(y, y_pred, marker=',', linestyle='None')
    # error
    #axs[1].plot(y, y_pred-y, marker='s', markeredgecolor = (0, 0, 0, 1), markerfacecolor = (1, 1, 1, .5), linestyle='None', ms=4)
    axs[1].plot(y, y_pred-y, marker=',', linestyle='None')
    
    # axis equal for top
    if (model_name == 'MLR with categorical') or (model_name == 'MLR Lasso') or (model_name == 'MLR added features') or (model_name == 'Decision Tree Regression'):
        axs[0].set_xscale('log')
        axs[0].set_yscale('log')
        axs[1].set_xscale('log')
    axs[0].set_aspect(1)
    # store limits
    yl = axs[0].get_ylim()
    xl_top = axs[0].get_xlim()
    xl_bot = axs[1].get_xlim()
    xl = [np.max([xl_top[0], xl_bot[0]]), np.min([xl_top[1], xl_bot[1]])]
    # plot unity line and 0 error
    unity_line = [np.max([xl[0], yl[0]]), np.min([xl[1], yl[1]])]
    axs[0].plot(unity_line, unity_line, '-k', linewidth=2)
    axs[1].plot(xl, [0, 0], '-k', linewidth=2)
    # reset limits
    axs[0].set_xlim(xl)
    axs[1].set_xlim(xl)

    # make equal size panels
    # Note: sharex did not work
    bb=axs[0].get_position(False)
    rect_top = bb.bounds
    bb=axs[1].get_position(False)
    rect_bot = bb.bounds
    rect = list(rect_bot)
    rect[0] = rect_top[0]
    rect[2] = rect_top[2]
    axs[1].set_position(rect)
    
    # labeling
    fig.suptitle('{}\nrmse: EUR {:.0f}'.format(model_name,rmse), style='italic')
    axs[1].set_xlabel('Real price ' + unit, style='italic')
    axs[0].set_ylabel('Predicted price\n' + unit, style='italic')
    axs[1].set_ylabel('Prediction error\n' + unit, style='italic')
    
    # save
    file_name = '../results/{}-accuracy.png'.format(model_name.replace(' ','_'))
    if (SKIPSAVE==False): #and (not(os.path.isfile(file_name))):
        print(file_name)
        with plt.style.context('../assets/context-paper.mplstyle'):
            plt.savefig(file_name, bbox_inches='tight', transparent=False)
    else:
        plt.show()
        print(f'Skip. {file_name} exists or saving is disabled in settings.')

linear regression no cv
9366.3985169183
../results/linear_regression_no_cv-accuracy.png
linear regression log price
9551.893168188062
../results/linear_regression_log_price-accuracy.png
linear regression log price young
8625.006854508629
../results/linear_regression_log_price_young-accuracy.png
MLR reduced observations
8914.656140122761
../results/MLR_reduced_observations-accuracy.png
MLR impute median
8977.799897302903
../results/MLR_impute_median-accuracy.png
MLR with categorical
9351.61363113215
../results/MLR_with_categorical-accuracy.png
MLR Lasso
7049.344819692632
../results/MLR_Lasso-accuracy.png
MLR added features
7672.922931224008
../results/MLR_added_features-accuracy.png


In [54]:
assert False, 'stop running, below is sandboxing and testing'

AssertionError: stop running, below is sandboxing and testing

<a href="#pred_top" id='pred_save_model'><font size=+1><center>^^ TOP ^^</center></font></a>

---

# Save model as pickle
Save the best model as a .pkl file.

See also: https://scikit-learn.org/stable/modules/model_persistence.html

In [ ]:
# import dill # dill acts as pickle but handles lambda functions
model_name = 'MLR Lasso' 
model = models[model_name]

In [ ]:
model['name'] = model_name
fn = '../results/trained_model_{}.pkl'.format(model_name.replace(' ', '_').lower())
print(fn)
# with open(fn, 'wb') as file:
#     dill.dump(model, file)

<a href="#pred_top" id='pred_predict'><font size=+1><center>^^ TOP ^^</center></font></a>

---

# Example predictions

In [ ]:
# Predict some known cars
B = pd.DataFrame(columns=X.columns, index=['Mine'])
B.loc['Mine', ['brand', 'model', 'fuel', 'body_type', 'color']] = ['CITROËN', 'berlingo', 'benzine', 'mpv', 'Gray']
B.loc['Mine', ['displacement', 'number_of_cylinders', 'number_of_seats', 'number_of_doors', 'fourwd', 'number_of_gears']] = [1600, 4, 5, 5, 'n', 5]
B.loc['Mine', ['top_speed']] = [170]
B.loc['Mine', 'age'] = (pd.Timestamp.now() - pd.to_datetime('2005-12-1')).days
B.loc['Mine', 'days_since_inspection_invalid'] = (pd.Timestamp.now() - pd.to_datetime('2022-6-11')).days
B.loc['Mine', 'age_at_import'] = 0
B.loc['Mine', 'odometer'] = 160000
B.loc['Mine', ['weight']] = [1326]

B.loc['Peer', ['brand', 'model', 'fuel', 'body_type', 'color']] = ['CITROËN', 'ax', 'benzine', 'hatchback', 'Gray']
B.loc['Peer', ['displacement', 'number_of_cylinders', 'number_of_seats', 'number_of_doors', 'fourwd', 'number_of_gears']] = [1100, 4, 5, 5, 'n', 5]
B.loc['Peer', ['top_speed']] = [170]
B.loc['Peer', 'age'] = (pd.Timestamp.now() - pd.to_datetime('1996-12-1')).days
# B.loc['Mine', 'days_since_inspection_invalid'] = (pd.Timestamp.now() - pd.to_datetime('2020-6-11')).days
B.loc['Peer', 'age_at_import'] = 0
B.loc['Peer', 'odometer'] = 160000
B.loc['Peer', ['weight']] = [800]

B.loc['a car', ['brand']] = [np.NaN]


B.loc['J-892-TZ', ['brand', 'model', 'fuel', 'body_type', 'color']] = ['SUZUKI', 'sx4', 'benzine', 'hatchback', 'Gray']
B.loc['J-892-TZ', ['displacement', 'number_of_cylinders', 'number_of_seats', 'number_of_doors', 'fourwd', 'number_of_gears']] = [1586, 4, 5, 5, 'n', np.NaN]
B.loc['J-892-TZ', 'age'] = (pd.Timestamp.now() - pd.to_datetime('2010-11-11')).days
B.loc['J-892-TZ', 'days_since_inspection_invalid'] = (pd.Timestamp.now() - pd.to_datetime('2021-11-18')).days
B.loc['J-892-TZ', 'age_at_import'] = (pd.Timestamp.now() - pd.to_datetime('2020-11-19')).days
B.loc['J-892-TZ', 'odometer'] = 58153
B.loc['J-892-TZ', 'weight'] = 1230
B.loc['J-892-TZ', 'power'] = 118
B.loc['J-892-TZ', 'automatic_gearbox'] = 'y'
B.loc['J-892-TZ', 'private_owners'] = 1
B.loc['J-892-TZ', 'company_owners'] = 0
B.loc['J-892-TZ', 'sale_price'] = 19979
B.loc['J-892-TZ', 'registration_tax'] = 3936

B.loc['J-892-TZ-real'] = df.loc['2022-01-805121',:].drop(columns='Price')

B.T

In [ ]:
df_ = pd.DataFrame(index=models.keys(), columns=B.index)
for model in df_.index[::-1]:
    try:
        print(f'{model}')
        B.loc[:,'predict'] = models[model]['model'].predict(B)
        pred = B.predict
    except: 
        pred = pd.Series(index=B.index, data=np.NaN)
    df_.loc[model,:] = pred
    
df_

In [ ]:
ix = df[df.model == 'sx4'].index
ix = df.loc[
    (df.brand == 'VOLKSWAGEN') & 
    (df.fuel == 'benzine') & 
    (df.number_of_cylinders == 5)
    ,:].index
B2 = df.loc[ix,:].drop(columns='price')
display(B2)
models['MLR Lasso']['model'].predict(B2)


In [ ]:
df.loc['2022-04-704304',:]

In [ ]:
B = pd.read_pickle('/home/tom/bin/satdatsci/Saturday-Datascience/data/rdw-data-2021-02.pkl')
B.columns
#['brand', 'model', 'fuel', 'body_type', 'color']
B.loc[:, [
    'rdw_merk',
    'rdw_type',
    'rdw_brandstof_brandstof_omschrijving_1',
    'rdw_ovi_inrichting_code_omschrijving',
    'rdw_eerste_kleur',
     ]]

In [ ]:
B.loc[:,(B == 'GRIJS').any()]

In [ ]:
m = models['MLR Lasso']['model']
m.n_features_in_

In [ ]:
p = m.get_params(deep=True)
t=p['regressor__steps'][0][1]
e = t.get_params(deep=True)

In [ ]:
input_columns = []
for estimator in e['transformers']:
    idx = estimator[2]
    if isinstance(idx, str):
        input_columns += [idx]
    else:
        input_columns += list(idx)

In [ ]:
len(input_columns)

In [ ]:
[c for c in X.columns.to_list() if c not in input_columns]